In [3]:
import pandas as pd

# ! options !
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)


In [4]:
## Import Data ##

chemin_fichier_2018 = '../valeursfoncieres-2018.txt'
chemin_fichier_2019 = '../valeursfoncieres-2019.txt'
chemin_fichier_2020 = '../valeursfoncieres-2020.txt'
chemin_fichier_2021 = '../valeursfoncieres-2021.txt'


df_18 = pd.read_csv(chemin_fichier_2018, delimiter='|',nrows=100000)
#df_19 = pd.read_csv(chemin_fichier_2019, delimiter='|')
#df_20 = pd.read_csv(chemin_fichier_2020, delimiter='|')
#df_21 = pd.read_csv(chemin_fichier_2021, delimiter='|')

/tmp/ipykernel_7652/2745004854.py:9: DtypeWarning: Columns (24,26,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df_18 = pd.read_csv(chemin_fichier_2018, delimiter='|',nrows=100000)


In [5]:
# Concat all years in one dataframe (and take some parts if we want)

df_18 = df_18.sample(frac=1, random_state=42)
#df_19 = df_19.sample(frac=1, random_state=42)
#df_20 = df_20.sample(frac=1, random_state=42)
# df_21 = df_21.sample(frac=1, random_state=42)

# df = pd.concat([df_18, df_19, df_20, df_21], axis=0)
df = pd.concat([df_18], axis=0)


In [6]:
# Replace ' ' by '_'

df.columns = df.columns.str.replace(' ', '_')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 75721 to 15795
Data columns (total 43 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Identifiant_de_document     0 non-null       float64
 1   Reference_document          0 non-null       float64
 2   1_Articles_CGI              0 non-null       float64
 3   2_Articles_CGI              0 non-null       float64
 4   3_Articles_CGI              0 non-null       float64
 5   4_Articles_CGI              0 non-null       float64
 6   5_Articles_CGI              0 non-null       float64
 7   No_disposition              100000 non-null  int64  
 8   Date_mutation               100000 non-null  object 
 9   Nature_mutation             100000 non-null  object 
 10  Valeur_fonciere             98532 non-null   object 
 11  No_voie                     45343 non-null   float64
 12  B/T/Q                       3197 non-null    object 
 13  Type_de_voi

In [8]:
# Affichez les pourcentages de NaN pour chaque colonne

pourcentages_nan = (df.isna().sum() / len(df)) * 100
print(pourcentages_nan)

Identifiant_de_document      100.00
Reference_document           100.00
1_Articles_CGI               100.00
2_Articles_CGI               100.00
3_Articles_CGI               100.00
4_Articles_CGI               100.00
5_Articles_CGI               100.00
No_disposition                 0.00
Date_mutation                  0.00
Nature_mutation                0.00
Valeur_fonciere                1.47
No_voie                       54.66
B/T/Q                         96.80
Type_de_voie                  58.98
Code_voie                      0.11
Voie                           0.11
Code_postal                    0.11
Commune                        0.00
Code_departement               0.00
Code_commune                   0.00
Prefixe_de_section            98.02
Section                        0.00
No_plan                        0.00
No_Volume                     99.84
1er_lot                       84.10
Surface_Carrez_du_1er_lot     95.25
2eme_lot                      96.29
Surface_Carrez_du_2eme_lot  

In [9]:
# Remove empty columns and keep only 'Vente' 

columns_to_remove = ["Identifiant_de_document", "Reference_document", "1_Articles_CGI", "2_Articles_CGI", "3_Articles_CGI", "4_Articles_CGI", "5_Articles_CGI",
                     "B/T/Q", "Prefixe_de_section", "No_Volume", "Type_local", "Identifiant_local", "Nature_culture_speciale",
                     "1er_lot", "Surface_Carrez_du_1er_lot", "2eme_lot", "Surface_Carrez_du_2eme_lot", "3eme_lot", "Surface_Carrez_du_3eme_lot", "4eme_lot",
                     "Surface_Carrez_du_4eme_lot", "5eme_lot", "Surface_Carrez_du_5eme_lot"]


df = df.drop(columns_to_remove, axis=1)
df = df[df['Nature_mutation'] == "Vente"] 
df = df.drop(columns=['Nature_mutation'])
    

pourcentages_nan = (df.isna().sum() / len(df)) * 100
print(pourcentages_nan)


No_disposition               0.00
Date_mutation                0.00
Valeur_fonciere              1.22
No_voie                     52.87
Type_de_voie                57.61
Code_voie                    0.01
Voie                         0.01
Code_postal                  0.02
Commune                      0.00
Code_departement             0.00
Code_commune                 0.00
Section                      0.00
No_plan                      0.00
Nombre_de_lots               0.00
Code_type_local             53.99
Surface_reelle_bati         54.10
Nombre_pieces_principales   54.10
Nature_culture              14.17
Surface_terrain             14.17
dtype: float64


In [10]:
# Create variable 'adresse' and remove variables use to build the adress

columns_for_adress = ["No_voie", "Type_de_voie", "Code_voie",
                      "Voie", "Code_commune", "Code_departement"]


df['Code_postal'] = df.groupby('Commune')['Code_postal'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))  # match les code postal vide avec les communes (bien)
    
df['No_voie'] = df['No_voie'].apply(lambda x: str(int(x)) if pd.notnull(
        x) else '').astype(str)  # change le type si non il y a une valeur sinon affiche rien

df['adresse'] = df['No_voie'] + ' ' + df['Type_de_voie'].astype(str) + ' ' + \
df['Code_voie'].astype(str) + ' ' + df['Voie'].astype(str) + ' ' + \
df['Code_postal'].astype(str) + ' ' + df['Commune']
    
    
# Après la création de l'adresse on remove les variables concatenées et les autres inutiles
df = df.drop(columns_for_adress, axis=1)


df.head()

,No_disposition,Date_mutation,Valeur_fonciere,Code_postal,Commune,Section,No_plan,Nombre_de_lots,Code_type_local,Surface_reelle_bati,Nombre_pieces_principales,Nature_culture,Surface_terrain,adresse
75721,1,24/11/2018,"1660000,00",3250.00,CHATEL-MONTAGNE,D,194,0,NaN,NaN,NaN,BR,16615.00,nan B004 LE BAIS 3250.0 CHATEL-MONTAGNE
80184,1,12/09/2018,"182500,00",3100.00,MONTLUCON,CH,396,0,4.00,75.00,0.00,S,307.00,136 AV 3330 DE LA REPUBLIQUE 3100.0 MONTLUCON
76699,2,14/02/2018,"2500,00",3100.00,MONTLUCON,BO,458,0,NaN,NaN,NaN,S,75.00,RUE 0570 BUFFON 3100.0 MONTLUCON
92991,1,29/10/2018,"138100,00",4400.00,BARCELONNETTE,AD,513,1,2.00,50.00,2.00,NaN,NaN,2 AV 0203 EMILE GRASSET 4400.0 BARCELONNETTE
76434,1,02/02/2018,"27000,00",3380.00,LA CHAPELAUDE,ZW,127,0,1.00,48.00,2.00,S,350.00,5386 nan B069 LA VALLAS 3380.0 LA CHAPELAUDE


In [11]:
# Remove duplicates (date_mutation et adresse)

df = df.drop_duplicates(subset=["Date_mutation", "adresse"])

# Variable 'adresse' useless now => remove
df = df.drop("adresse", axis=1)

df.head()

,No_disposition,Date_mutation,Valeur_fonciere,Code_postal,Commune,Section,No_plan,Nombre_de_lots,Code_type_local,Surface_reelle_bati,Nombre_pieces_principales,Nature_culture,Surface_terrain
75721,1,24/11/2018,"1660000,00",3250.00,CHATEL-MONTAGNE,D,194,0,NaN,NaN,NaN,BR,16615.00
80184,1,12/09/2018,"182500,00",3100.00,MONTLUCON,CH,396,0,4.00,75.00,0.00,S,307.00
76699,2,14/02/2018,"2500,00",3100.00,MONTLUCON,BO,458,0,NaN,NaN,NaN,S,75.00
92991,1,29/10/2018,"138100,00",4400.00,BARCELONNETTE,AD,513,1,2.00,50.00,2.00,NaN,NaN
76434,1,02/02/2018,"27000,00",3380.00,LA CHAPELAUDE,ZW,127,0,1.00,48.00,2.00,S,350.00


In [12]:
# Affichez le nombre d'individus par modalité de la colonne 'Code_type_local'

comptage_modalites = df['Code_type_local'].value_counts()
print(comptage_modalites)


1.00    16071
2.00     5149
3.00     3537
4.00     1754
Name: Code_type_local, dtype: int64


In [13]:
print("before : ", len(df))

#df_modified = df_modified.dropna() # on ne fait plus ça

df['Valeur_fonciere'] = df['Valeur_fonciere'].astype(str).str.replace(',', '.').astype(float)  
    
print("after : ", len(df), "\n")

comptage_modalites = df['Code_type_local'].value_counts()
print(comptage_modalites)


before :  50169
after :  50169 

1.00    16071
2.00     5149
3.00     3537
4.00     1754
Name: Code_type_local, dtype: int64


In [14]:
masque_nan = df.isna()

# Utilisez sum() pour compter le nombre de valeurs manquantes (True) dans chaque colonne
nombre_nan_par_colonne = masque_nan.sum()
print(nombre_nan_par_colonne)

No_disposition                   0
Date_mutation                    0
Valeur_fonciere                429
Code_postal                      0
Commune                          0
Section                          0
No_plan                          0
Nombre_de_lots                   0
Code_type_local              23658
Surface_reelle_bati          23726
Nombre_pieces_principales    23726
Nature_culture                8010
Surface_terrain               8010
dtype: int64


In [15]:
# Remove individus with at least one NaN and outliers

print("before : ", len(df))

# OUTLIERS 
df['Valeur_fonciere'] = df['Valeur_fonciere'].astype(str).str.replace(',', '.').astype(float)

df = df.dropna(subset=['Valeur_fonciere'])
Q1 = df['Valeur_fonciere'].quantile(0.25)
Q3 = df['Valeur_fonciere'].quantile(0.75)
    
#print(Q1)
#print(Q3)

IQR = Q3 - Q1

# Définissez les limites supérieure et inférieure pour détecter les outliers
limite_inferieure = Q1 - 0.3 * IQR
limite_superieure = Q3 + 1.7 * IQR

print(limite_inferieure)
print(limite_superieure,"\n")
    
df = df[(df['Valeur_fonciere'] >= limite_inferieure) & (df['Valeur_fonciere'] <= limite_superieure)]
    
print("after : ", len(df), "\n")

comptage_modalites = df['Code_type_local'].value_counts()
print(comptage_modalites)


before :  50169
-12447.805
430479.245 

after :  47191 

1.00    15170
2.00     4930
3.00     3379
4.00     1497
Name: Code_type_local, dtype: int64


In [16]:
# Créer la variable "day", "month" and "year" 

# NB : Utile si on la mets en 'int' et pas en objet, car le prix de l'immobilier augmente chaque année, donc plus l'année augmente plus le prix aussi
# alors qu'en 'objet' inutile car dans ce sont les data de 2022 qui vont être testés (à revoir quand même)


df['Date_mutation'] = pd.to_datetime(df['Date_mutation'], format='%d/%m/%Y')

# Extract the year component and create a new 'year' column
df['day'] = df['Date_mutation'].dt.day
df['day'] = df['day'].astype(object)

df['month'] = df['Date_mutation'].dt.month
df['month'] = df['month'].astype(object)

df['year'] = df['Date_mutation'].dt.year
df['year'] = df['year'].astype(object)

df = df.drop("Date_mutation", axis=1)
    

df.head()


,No_disposition,Valeur_fonciere,Code_postal,Commune,Section,No_plan,Nombre_de_lots,Code_type_local,Surface_reelle_bati,Nombre_pieces_principales,Nature_culture,Surface_terrain,day,month,year
80184,1,182500.00,3100.00,MONTLUCON,CH,396,0,4.00,75.00,0.00,S,307.00,12,9,2018
76699,2,2500.00,3100.00,MONTLUCON,BO,458,0,NaN,NaN,NaN,S,75.00,14,2,2018
92991,1,138100.00,4400.00,BARCELONNETTE,AD,513,1,2.00,50.00,2.00,NaN,NaN,29,10,2018
76434,1,27000.00,3380.00,LA CHAPELAUDE,ZW,127,0,1.00,48.00,2.00,S,350.00,2,2,2018
84004,1,196550.00,3700.00,BRUGHEAS,YB,24,0,1.00,116.00,5.00,AG,514.00,25,9,2018


In [17]:
# Drop 'Code_postal' NaN anc change type

df = df.dropna(subset=['Code_postal'])
df['Code_postal'] = df['Code_postal'].astype(int)
df['Code_postal'] = df['Code_postal'].astype(object)  # puis en objet

print(df.info())
df.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 47191 entries, 80184 to 60263
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   No_disposition             47191 non-null  int64  
 1   Valeur_fonciere            47191 non-null  float64
 2   Code_postal                47191 non-null  object 
 3   Commune                    47191 non-null  object 
 4   Section                    47191 non-null  object 
 5   No_plan                    47191 non-null  int64  
 6   Nombre_de_lots             47191 non-null  int64  
 7   Code_type_local            24976 non-null  float64
 8   Surface_reelle_bati        24937 non-null  float64
 9   Nombre_pieces_principales  24937 non-null  float64
 10  Nature_culture             39473 non-null  object 
 11  Surface_terrain            39473 non-null  float64
 12  day                        47191 non-null  object 
 13  month                      47191 non-null 

,No_disposition,Valeur_fonciere,Code_postal,Commune,Section,No_plan,Nombre_de_lots,Code_type_local,Surface_reelle_bati,Nombre_pieces_principales,Nature_culture,Surface_terrain,day,month,year
80184,1,182500.00,3100,MONTLUCON,CH,396,0,4.00,75.00,0.00,S,307.00,12,9,2018
76699,2,2500.00,3100,MONTLUCON,BO,458,0,NaN,NaN,NaN,S,75.00,14,2,2018
92991,1,138100.00,4400,BARCELONNETTE,AD,513,1,2.00,50.00,2.00,NaN,NaN,29,10,2018
76434,1,27000.00,3380,LA CHAPELAUDE,ZW,127,0,1.00,48.00,2.00,S,350.00,2,2,2018
84004,1,196550.00,3700,BRUGHEAS,YB,24,0,1.00,116.00,5.00,AG,514.00,25,9,2018


In [18]:
# Load longitude, latitude, 'Code_postal' dataframe

geo = pd.read_csv('../data/communes-departement-region.csv')

geo.head()


,code_commune_INSEE,nom_commune_postal,code_postal,libelle_acheminement,ligne_5,latitude,longitude,code_commune,article,nom_commune,nom_commune_complet,code_departement,nom_departement,code_region,nom_region
0,1001,L ABERGEMENT CLEMENCIAT,1400,L ABERGEMENT CLEMENCIAT,NaN,46.15,4.93,1.00,L',Abergement-Clémenciat,L'Abergement-Clémenciat,1,Ain,84.00,Auvergne-Rhône-Alpes
1,1002,L ABERGEMENT DE VAREY,1640,L ABERGEMENT DE VAREY,NaN,46.01,5.43,2.00,L',Abergement-de-Varey,L'Abergement-de-Varey,1,Ain,84.00,Auvergne-Rhône-Alpes
2,1004,AMBERIEU EN BUGEY,1500,AMBERIEU EN BUGEY,NaN,45.96,5.37,4.00,NaN,Ambérieu-en-Bugey,Ambérieu-en-Bugey,1,Ain,84.00,Auvergne-Rhône-Alpes
3,1005,AMBERIEUX EN DOMBES,1330,AMBERIEUX EN DOMBES,NaN,46.00,4.91,5.00,NaN,Ambérieux-en-Dombes,Ambérieux-en-Dombes,1,Ain,84.00,Auvergne-Rhône-Alpes
4,1006,AMBLEON,1300,AMBLEON,NaN,45.75,5.59,6.00,NaN,Ambléon,Ambléon,1,Ain,84.00,Auvergne-Rhône-Alpes


In [19]:
geo.rename(columns={'code_postal': 'Code_postal'}, inplace=True)
geo.rename(columns={'nom_commune_postal': 'Commune'}, inplace=True)
geo.rename(columns={'code_departement': 'Code_departement'}, inplace=True)

In [20]:
# voir les doublons

doublons = geo[geo.duplicated(subset=['Code_postal', 'Commune'], keep=False)]
# OK -> supprmime les doublons Code postal => Commune, car quelques fois il y a une 'petite' commune dedans la grande enfin bref
geo = geo.drop_duplicates(subset=['Code_postal', 'Commune'])


In [21]:

geo['Commune'] = geo['Commune'].str.replace('-', ' ') 
df['Commune'] = df['Commune'].str.replace('-', ' ')
geo['Commune'] = geo['Commune'].str.replace('ST', 'SAINT')  # format 'ST' et 'SAINT'
df['Commune'] = df['Commune'].str.replace('ST', 'SAINT')
geo['Commune'] = geo['Commune'].str.replace('\'', ' ')
df['Commune'] = df['Commune'].str.replace('\'', ' ')

geo_commune = geo[['Commune', 'Code_postal', 'latitude', 'longitude']]


# MERGE
df = pd.merge(df, geo_commune, on=['Commune', 'Code_postal'], how='left')  # OK -> merge la commune et le code postal, car desfois la commune est dans différent département 


# Le reste des communes non trouvé c'est des erreurs dans le fichier geo, exemple => LA SALLE DES ALPES, dans les data => "LA SALLE", IMPOSSIBLE À gérer ce genre d'erreur humaine => à enlever
print(df['longitude'].isna().sum()," individus avec un nom de commune pas trouvé, on les enlève du dataset")

# Drop not found
df = df.dropna(subset=['longitude'])

print("nombre final d'individus : ",len(df))

df.head(20)


552  individus avec un nom de commune pas trouvé, on les enlève du dataset
nombre final d'individus :  46639


,No_disposition,Valeur_fonciere,Code_postal,Commune,Section,No_plan,Nombre_de_lots,Code_type_local,Surface_reelle_bati,Nombre_pieces_principales,Nature_culture,Surface_terrain,day,month,year,latitude,longitude
0,1,182500.00,3100,MONTLUCON,CH,396,0,4.00,75.00,0.00,S,307.00,12,9,2018,46.34,2.60
1,2,2500.00,3100,MONTLUCON,BO,458,0,NaN,NaN,NaN,S,75.00,14,2,2018,46.34,2.60
2,1,138100.00,4400,BARCELONNETTE,AD,513,1,2.00,50.00,2.00,NaN,NaN,29,10,2018,44.38,6.65
3,1,27000.00,3380,LA CHAPELAUDE,ZW,127,0,1.00,48.00,2.00,S,350.00,2,2,2018,46.43,2.50
4,1,196550.00,3700,BRUGHEAS,YB,24,0,1.00,116.00,5.00,AG,514.00,25,9,2018,46.07,3.37
5,1,2419.00,2310,PAVANT,C,168,0,NaN,NaN,NaN,VE,122.00,23,2,2018,48.95,3.28
6,1,87500.00,2100,SAINT QUENTIN,AE,33,0,2.00,39.00,1.00,S,135.00,19,12,2018,49.85,3.28
7,1,519.00,1200,INJOUX GENISSIAT,C,9,0,NaN,NaN,NaN,T,1730.00,11,12,2018,46.05,5.76
8,1,10000.00,2860,BRUYERES ET MONTBERAULT,A,131,0,NaN,NaN,NaN,AG,2716.00,27,7,2018,49.52,3.67
9,1,80000.00,2320,ANIZY LE CHATEAU,AC,21,0,4.00,50.00,0.00,S,90.00,22,11,2018,49.50,3.43


In [22]:
# Handle 'Corse' Département

def assign_department(code_postal):
    if 20000 <= code_postal <= 20190:
        return '2A' # Pour gérer la corse sud
    elif 20200 <= code_postal <= 20270:
        return '2B' # Pour gérer la corse nord
    elif len(str(code_postal)) == 4:
        dep = '0' + str(code_postal)[:1] # Pour gérer les 10 premiers département
        return dep
    else:
        dep = str(code_postal)[:2]
        return dep 


# Appliquez la fonction à la colonne 'Code_postal' en utilisant apply()
df['Departement'] = df['Code_postal'].apply(lambda x: assign_department(x))
df['Departement'] = df['Departement'].astype(str)

In [23]:
# Create a dataframe save with the Nan, before dropping them

df_sauvegarde_with_nan = df.copy()

# exporter DF
df_sauvegarde_with_nan.to_pickle('../data/df_with_nan.pkl')


In [24]:
# Drop the Nan remainings, for the model

df = df.dropna()
df['Code_type_local'] = df['Code_type_local'].astype(int)  

# exporter DF
df.to_pickle('../data/df_without_nan.pkl')
